In [1]:
map_rels_no = {'COM':0, 'CONTR':1, 'CORR':2, 'QAP':3, 'ACK':4,'ELAB':5,
                 'CLARIFQ':6, 'COND':7, 'CONTIN':8, 'RES':9, 'EXPL':10, 'QELAB':11,
                 'ALT':12, 'NARR':13, 'CONFQ':14, 'SEQ':15, 'NULL':16}

# reverse_relations = {0:'COM', 1: 'CONTR', 2 :'CORR', 3: 'QAP', 4: 'ACK', 5: 'ELAB',
#             6: 'CLARIFQ', 7: 'COND', 8: 'CONTIN', 9: 'RES', 10: 'EXPL',
#             11: 'QELAB', 12: 'ALT', 13: 'NARR', 14: 'CONFQ', 15: 'SEQ', 16: 'NULL'}

reverse_relations = {'Comment':0, 'Contrast':1, 'Correction':2, 'Question-answer_pair':3, 'Acknowledgement':4,'Elaboration':5,
                 'Clarification_question':6, 'Conditional':7, 'Continuation':8, 'Result':9, 'Explanation':10, 'Q-Elab':11,
                 'Alternation':12, 'Narration':13, 'Confirmation_question':14, 'Sequence':15, 'Break':16}


In [40]:
map_rels_str = {'Comment':'COM', 'Contrast':'CONTR', 'Correction':'CORR', 'Question-answer_pair':'QAP', 'Acknowledgement':'ACK','Elaboration':'ELAB',
                 'Clarification_question':'CLARIFQ', 'Conditional':'COND', 'Continuation':'CONTIN', 'Result':'RES', 'Explanation':'EXPL', 'Q-Elab':'QELAB',
                 'Alternation':'ALT', 'Narration':'NARR', 'Background':'BACK', 'Parallel':'PAR', 'Sequence':'SEQ', 'Question_answer_pair':'QAP',  
                 'Q_Elab':'QELAB', 'Confirmation_question' : 'CONFQ'}


STAC LINGUISTIC TEST

game pilot02_fix9 has only one edu
game s2-leagueM-game4_fix14 has only one edu

s1-league3-game3_fix24: [2], --> 2-->COMMENT-->3
s2-league4-game2_fix5: [4],  --> 4---> COMMENT --> 5
s2-league4-game2_fix17: [7], --->7 --> QAP --> 8
s2-league4-game2_fix23: [3]  ** NO RELATION


MSDC 
defaultdict(list,
            {'C70-B11-A7': [60],
             'C105-B35-A39': [38],
             'C101-B44-A36': [35],
             'C86-B16-A26': [11],
             'C70-B29-A26': [54],
             'C111-B35-A44': [14],
             'C52-B36-A38': [8]})


## STAC counts

In [53]:
#STAC COUNTS real quick
import json
from collections import Counter, defaultdict
home=%pwd
stac_path = home + '/stac_linguistic_only.json'
save_path = home + '/stac_linguistic_flat.json'


In [7]:
with open(stac_path, 'r') as j:
    jfile = json.load(j)
    data = jfile

In [8]:
len(data['dialogues'])

1137

In [36]:
data['dialogues'][0].keys()

dict_keys(['dialogue_id', 'edus', 'cdus', 'relations'])

In [58]:
probdial = [d for d in data['dialogues'] if d['dialogue_id'] == '331_s1_league1_game4_1'][0]

In [60]:
heads_dict = {}
#first get a list of CDUs contain no other CDUS
all_cdus = [c['cdu_id'] for c in probdial['cdus']]
edu_spans = {e['seg_id']: e['span_end'] for e in probdial['edus']}
compounds = []
for cdu in probdial['cdus']:
    g = set(cdu['members']).intersection(all_cdus)
    if len(g) != 0:
        compounds.append(cdu)
    else:
        spans = [(member, edu_spans[member]) for member in cdu['members']]
        heads = sorted(spans, key=lambda x: x[1])
        head = heads[0][0]
        heads_dict[cdu['cdu_id']] = head
##then go through and get the compound edu heads
compounds_level_2 = []
if len(compounds) > 0:
    for cdu in compounds:
        spans = []
        for member in cdu['members']:
            if member in edu_spans.keys():
                spans.append((member, edu_spans[member]))
                heads = sorted(spans, key=lambda x: x[1])
                head = heads[0][0]
                heads_dict[cdu['cdu_id']] = head
            elif member in heads_dict.keys():
                new_id = heads_dict[member]
                spans.append((new_id, edu_spans[new_id]))
                heads = sorted(spans, key=lambda x: x[1])
                head = heads[0][0]
                heads_dict[cdu['cdu_id']] = head
            else:
                compounds_level_2.append(cdu)
if len(compounds_level_2) > 1:
    for cdu in compounds_level_2:
        spans = []
        for member in cdu['members']:
            try:
                spans.append((member, edu_spans[member]))
            except KeyError:
                new_id = heads_dict[member]
                spans.append((new_id, edu_spans[new_id]))

            heads = sorted(spans, key=lambda x: x[1])
            head = heads[0][0]
            heads_dict[cdu['cdu_id']] = head

#make sure you have all the 
assert len(heads_dict) == len(all_cdus)

### FLATTEN ALL DATA

In [64]:
def get_new_dialogue(test_dial):
    """
    takes a full stac dialogue and returns a flattened and 
    reformatted dialogue
    """
    #rename cdus
    heads_dict = {}
    #first get a list of CDUs contain no other CDUS
    all_cdus = [c['cdu_id'] for c in test_dial['cdus']]
    edu_spans = {e['seg_id']: e['span_end'] for e in test_dial['edus']}
    compounds = []
    for cdu in test_dial['cdus']:
        g = set(cdu['members']).intersection(all_cdus)
        if len(g) != 0:
            compounds.append(cdu)
        else:
            spans = [(member, edu_spans[member]) for member in cdu['members']]
            heads = sorted(spans, key=lambda x: x[1])
            head = heads[0][0]
            heads_dict[cdu['cdu_id']] = head
    ##then go through and get the compound edu heads
    compounds_level_2 = []
    if len(compounds) > 0:
        for cdu in compounds:
            spans = []
            for member in cdu['members']:
                if member in edu_spans.keys():
                    spans.append((member, edu_spans[member]))
                    heads = sorted(spans, key=lambda x: x[1])
                    head = heads[0][0]
                    heads_dict[cdu['cdu_id']] = head
                elif member in heads_dict.keys():
                    new_id = heads_dict[member]
                    spans.append((new_id, edu_spans[new_id]))
                    heads = sorted(spans, key=lambda x: x[1])
                    head = heads[0][0]
                    heads_dict[cdu['cdu_id']] = head
                else:
                    compounds_level_2.append(cdu)
    if len(compounds_level_2) > 1:
        for cdu in compounds_level_2:
            spans = []
            for member in cdu['members']:
                try:
                    spans.append((member, edu_spans[member]))
                except KeyError:
                    new_id = heads_dict[member]
                    spans.append((new_id, edu_spans[new_id]))

                heads = sorted(spans, key=lambda x: x[1])
                head = heads[0][0]
                heads_dict[cdu['cdu_id']] = head

    #make sure you have all the 
    assert len(heads_dict) == len(all_cdus)
    #now that you have the heads dict, you can fix the relations
    #get a dict of {seg id : index}
    segs = sorted(edu_spans.items(), key=lambda x : x[1])
    seg_indexes = {s[0]:ind for ind, s in enumerate(segs)}
    #need to add cdu ids!
    cdu_indexes = [(h[0], seg_indexes[h[1]]) for h in heads_dict.items()]
    for ci in cdu_indexes:
        seg_indexes[ci[0]] = ci[1]#now you have a dict with ALL edu/cdu ids and their indexes

    #make new dialogue
    new_dialogue = {}
    new_dialogue['dialogue_id'] = test_dial['dialogue_id']
    #add edus in orderdata['dialogues']
    new_edus = []
    for edu in segs:
        new_edu = {}
        old_edu = [e for e in test_dial['edus'] if e['seg_id'] == edu[0]][0]
        new_edu['speaker'] = old_edu['speaker']
        new_edu['text'] = old_edu['text']
        new_edus.append(new_edu)
    new_dialogue['edus'] = new_edus
    #add relations 
    new_relations = []
    for rel in test_dial['relations']:
        new_rel = {}
        new_rel['x'] = seg_indexes[rel['source']]
        new_rel['y'] = seg_indexes[rel['target']]
        new_rel['type'] = rel['type']
        new_relations.append(new_rel)
    new_dialogue['relations'] = new_relations

    assert len(new_dialogue['edus']) == len(test_dial['edus'])
    assert len(new_dialogue['relations']) == len(test_dial['relations'])

    return new_dialogue


In [65]:
new_flat_lingiustic = {}
new_flat_lingiustic['data_id'] = 'stac linguistic'
new_flat_dialogues = []
for dlg in data['dialogues']:
    if len(dlg['edus']) > 1:
        print(dlg['dialogue_id'])
        reformat = get_new_dialogue(dlg)
        new_flat_dialogues.append(reformat)
new_flat_lingiustic['dialogues'] = new_flat_dialogues
        

1_pilot01_1
2_pilot01_2
3_pilot01_3
4_pilot01_4
5_pilot01_5
6_pilot01_6
7_pilot01_7
8_pilot01_8
9_pilot01_9
10_pilot01_10
11_pilot01_11
12_pilot01_12
13_pilot01_13
14_pilot01_14
15_pilot01_15
16_pilot01_16
17_pilot01_17
18_pilot01_18
19_pilot01_19
20_pilot01_20
21_pilot01_21
22_pilot01_22
23_pilot01_23
24_pilot01_24
25_pilot01_25
26_pilot01_26
27_pilot01_27
28_pilot01_28
29_pilot01_29
30_pilot01_30
31_pilot01_31
32_pilot01_32
33_pilot01_33
34_pilot01_34
35_pilot01_35
36_pilot01_36
37_pilot01_37
38_pilot01_38
39_pilot01_39
40_pilot01_40
41_pilot01_41
42_pilot01_42
43_pilot01_43
44_pilot01_44
45_pilot01_45
46_pilot02_1
47_pilot02_2
48_pilot02_3
49_pilot02_4
50_pilot02_5
51_pilot02_6
52_pilot02_7
53_pilot02_8
54_pilot02_9
55_pilot02_10
56_pilot02_11
57_pilot02_12
58_pilot02_13
59_pilot02_14
60_pilot02_15
61_pilot02_16
62_pilot02_17
63_pilot02_18
65_pilot02_20
66_pilot02_21
67_pilot02_22
68_pilot02_23
69_pilot02_24
70_pilot02_25
71_pilot03_1
72_pilot03_2
73_pilot03_3
74_pilot03_4
75_pilot0

In [66]:
#save the new json
with open(save_path, 'w') as outfile:
    json.dump(new_flat_lingiustic, outfile)

In [73]:
flat_path = home + '/stac_linguistic_flat.json'
flat_train = home + '/stac_linguistic_flat_train.json'
flat_test = home + '/stac_linguistic_flat_test.json'

In [85]:
#make train-test split
test_games = ['s1_league3_game3', 's2_league4_game2','s2_leagueM_game4','pilot02']

In [86]:
with open(flat_path, 'r') as j:
    jfile = json.load(j)
    data = jfile['dialogues']

In [91]:
train_data = []
test_data = []
for dial in data:
    for t in test_games:
        if t in dial['dialogue_id']:
            test_data.append(dial)
            break
    else:
        train_data.append(dial)

print("train: ", len(train_data))
print("test: ", len(test_data))

train:  992
test:  109


In [94]:
for t in train_data:
    print(t['dialogue_id'])

1_pilot01_1
2_pilot01_2
3_pilot01_3
4_pilot01_4
5_pilot01_5
6_pilot01_6
7_pilot01_7
8_pilot01_8
9_pilot01_9
10_pilot01_10
11_pilot01_11
12_pilot01_12
13_pilot01_13
14_pilot01_14
15_pilot01_15
16_pilot01_16
17_pilot01_17
18_pilot01_18
19_pilot01_19
20_pilot01_20
21_pilot01_21
22_pilot01_22
23_pilot01_23
24_pilot01_24
25_pilot01_25
26_pilot01_26
27_pilot01_27
28_pilot01_28
29_pilot01_29
30_pilot01_30
31_pilot01_31
32_pilot01_32
33_pilot01_33
34_pilot01_34
35_pilot01_35
36_pilot01_36
37_pilot01_37
38_pilot01_38
39_pilot01_39
40_pilot01_40
41_pilot01_41
42_pilot01_42
43_pilot01_43
44_pilot01_44
45_pilot01_45
71_pilot03_1
72_pilot03_2
73_pilot03_3
74_pilot03_4
75_pilot03_5
76_pilot03_6
77_pilot03_7
78_pilot03_8
79_pilot03_9
80_pilot03_10
81_pilot03_11
82_pilot03_12
83_pilot03_13
84_pilot03_14
85_pilot03_15
86_pilot03_16
87_pilot03_17
88_pilot03_18
89_pilot03_19
90_pilot03_20
91_pilot03_21
92_pilot03_22
93_pilot03_23
94_pilot03_24
95_pilot03_25
96_pilot03_26
97_pilot03_27
98_pilot03_28
99_pi

In [93]:
with open(flat_train, 'w') as outfile:
    json.dump(train_data, outfile)
with open(flat_test, 'w') as outfile:
    json.dump(test_data, outfile)

#### backwards relations

In [59]:
bkwds = defaultdict(list)
for d in data:
    for rel in d['relations']:
        if rel['y'] < rel['x']:
            bkwds[map_rels_str[rel['type']]].append(rel['y'] - rel['x'])

In [60]:
for k in bkwds.keys():
    print('{} rel : {}'.format(k, len(bkwds[k])))

COM rel : 147
COND rel : 7


STAC SQUISHED
COND rel : 68
COM rel : 57
EXPL rel : 4
BACK rel : 10
ELAB rel : 3

STAC LINGUISTIC
COND rel : 67
CONTIN rel : 1
COM rel : 61
BACK rel : 8
ELAB rel : 3
EXPL rel : 8

In [ ]:
edus = 0
eeus = 0
for d in stac['dialogues']:
    total_dus = len(d['edus'])
    eeu_count = len([e for e in d['edus'] if e['speaker'] in ['UI', 'Server']])
    eeus += eeu_count
    edu_count = total_dus - eeu_count
    edus += edu_count

print("edus: ", edus)
print("eeus: ", eeus)

In [ ]:
cdus = 0
for d in stac['dialogues']:
    cdus += len(d['cdus'])

print("cdus: ", cdus)


In [ ]:
rels = 0
for d in stac['dialogues']:
    rels += len(d['relations'])

print("rels: ", rels)

In [ ]:
mpdu = 0
for d in stac['dialogues']:
    
    rels_count = Counter([g['target'] for g in d['relations']])
    mps = [r[0] for r in rels_count.items() if r[1] > 1] 
    mpdu += len(mps)

print("mpdus: ", mpdu)